---
title: DeepHyperGraph
subtitle: How to use DHG; top-down sketch from official documents
description: "deep learning library built upon PyTorch for learning with both Graph Neural Networks and Hypergraph Neural Networks"
categories:
  - HAR/frameworks
author: YeEun Hong
date: 2023-03-25
---

### installation guide
- [docs/install](https://deephypergraph.readthedocs.io/en/0.9.3/start/install.html)


```bash
# conda env : python 3.8
! pip install dhg

# dependencies / requirements
! conda install -c pytorch pytorch    # (최신버전 설치) dhg는 torch를 확장한 패키지, (23.03 현재 m1 가속을 지원한다.)
! pip install tqdm  # dhg에서 요구
```

In [1]:
# for M1 MAC / pytorch check

import torch; device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

print(f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS supportive build? : {torch.backends.mps.is_built()}")
print(f"MPS available : {torch.backends.mps.is_available()}")
!python -c 'import platform;print(platform.platform())'

PyTorch version:2.0.0
MPS supportive build? : True
MPS available : True
macOS-13.0-arm64-arm-64bit


### Cooking200 Example (Vertex Classification)

> The Cooking 200 dataset (dhg.data.Cooking200) is collected from Yummly.com for vertex classification task. It is a hypergraph dataset, in which vertex denotes the dish and hyperedge denotes the ingredient. Each dish is also associated with category information, which indicates the dish’s cuisine like Chinese, Japanese, French, and Russian.

#### intro
graph dataset은 cora, pubmed 등 논문과 논문의 저자 관계를 다룬 데이터셋을 사용하는게 일반적이지만 hypergraph dataset은 그보다 조금 더 복잡한, 다수의 관계를 매핑한 데이터셋을 사용한다. `Yummly` 에서 제작한 데이터셋인 Cooking200의 구조는 아래와 같다.
- vertex (=node) : dish, receipt
- hyperedge (node 간의 관계) : ingredient
    - hyperedge: A connection between two or more vertices of a hypergraph. A hyperedge connecting just two vertices is simply a usual graph edge.
    - G는 결국 HG의 특수한 형태로, hyperedge를 edge의 확장으로 생각하는 것 보다 hyperedge의 특수한 경우를 edge로 이해하는 편이 빠르다. 결국 **edge는 node간의 관계를 의미하는 용어**이고 일반 graph에서 edge는 1:1 관계, hypergraph에서의 edge, 즉 hyperedge는 다수간의 연결을 의미한다.

#### Cooking200

Cooking200은 Kaggle [https://www.kaggle.com/c/whats-cooking](https://www.kaggle.com/c/whats-cooking) 에서 공개된 데이터셋이다. CLI환경에서는 `kaggle competitions download -c whats-cooking` 으로 다운로드 가능하며 train.json은 id:int, cuisine:int,ingredients:list 구조로 구성되어 있는데, 각 재료(ingredient)는 한가지 요리(dish)에 종속되지 않으며 각 분류(cuisine) 또한 하나의 요리에 종속되지 않는다. 따라서 node인 재료와 요리의 관계는 재료와 분류의 관계와 일치하지 않는다.
- 이걸 어떻게 설명해야하지 문장으로...

#### NOTE
1. The dataset is a hypergraph dataset, which cannot be directly used for GCN model. Thus, `the clique expansion is adpoted` to reduce the hypergraph structure to a graph structure.
    - GCN 구조상 A를 만들지 않고서는 GCN에 hypergraph dataset을 사용할 수 없다.
    - expansion의 경우 `dhg` 에서 제공하는 기능으로 구현할 수 있는데 이는 다음 [공식문서](https://deephypergraph.readthedocs.io/en/latest/tutorial/structure.html#reduced-from-high-order-structures)를 참고 할 것. star expansion, clique expansion, hypergcn* 의 방법을 지원한다.
        - *hypergcn : ('In the HyperGCN paper, the authors also describe a method to reduce the hyperedges in the hypergraph to the edges in the graph as the following figure.')
        ![hypergraph from hypergcn](https://deephypergraph.readthedocs.io/en/latest/_images/hypergcn.png)
2. The dataset `do not contain the vertex features`. Thus, we generate a identity matrix for vertex features.

In [2]:
# Example : https://deephypergraph.readthedocs.io/en/latest/examples/vertex_cls/hypergraph.html#gcn-on-cooking200

'''Import libraries'''

import time
from copy import deepcopy

import torch
import torch.optim as optim
import torch.nn.functional as F

from dhg import Graph, Hypergraph
from dhg.data import Cooking200
from dhg.models import GCN
from dhg.random import set_seed
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator

'''Define Functions'''

def train(net, X, A, lbls, train_idx, optimizer, epoch):
    net.train()

    st = time.time()
    optimizer.zero_grad()
    outs = net(X, A)
    outs, lbls = outs[train_idx], lbls[train_idx]
    loss = F.cross_entropy(outs, lbls)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Time: {time.time()-st:.5f}s, Loss: {loss.item():.5f}")
    return loss.item()


@torch.no_grad()
def infer(net, X, A, lbls, idx, test=False):
    net.eval()
    outs = net(X, A)
    outs, lbls = outs[idx], lbls[idx]
    if not test:
        res = evaluator.validate(lbls, outs)
    else:
        res = evaluator.test(lbls, outs)
    return res

'''Main'''
if __name__ == "__main__":
    set_seed(2021)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])
    data = Cooking200()

    X, lbl = torch.eye(data["num_vertices"]), data["labels"]
    ft_dim = X.shape[1]
    HG = Hypergraph(data["num_vertices"], data["edge_list"])
    G = Graph.from_hypergraph_clique(HG, weighted=True)
    train_mask = data["train_mask"]
    val_mask = data["val_mask"]
    test_mask = data["test_mask"]

    net = GCN(ft_dim, 32, data["num_classes"], use_bn=True)
    optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

    X, lbl = X.to(device), lbl.to(device)
    G = G.to(device)
    net = net.to(device)

    best_state = None
    best_epoch, best_val = 0, 0
    for epoch in range(3):        # 200 epoch -> 3 으로 줄여서 테스트
        # train
        train(net, X, G, lbl, train_mask, optimizer, epoch)
        # validation
        if epoch % 1 == 0:
            with torch.no_grad():
                val_res = infer(net, X, G, lbl, val_mask)
            if val_res > best_val:
                print(f"update best: {val_res:.5f}")
                best_epoch = epoch
                best_val = val_res
                best_state = deepcopy(net.state_dict())
    print("\ntrain finished!")
    print(f"best val: {best_val:.5f}")
    # test
    print("test...")
    net.load_state_dict(best_state)
    res = infer(net, X, G, lbl, test_mask, test=True)
    print(f"final result: epoch: {best_epoch}")
    print(res)

/opt/homebrew/Caskroom/miniforge/base/envs/hypergraph-py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/hypergraph-py38/lib/python3.8/site-packages/dhg/structure/graphs/graph.py:247: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1678455016227/work/aten/src/ATen/SparseCsrTensorImpl.cpp:56.)
  adj = miu * hypergraph.H.mm(hypergraph.H_T).coalesce().cpu().clone()


Epoch: 0, Time: 7.83699s, Loss: 3.02474
update best: 0.05000
Epoch: 1, Time: 0.22393s, Loss: 2.47472
Epoch: 2, Time: 0.21954s, Loss: 2.41068

train finished!
best val: 0.05000
test...
final result: epoch: 0
{'accuracy': 0.01942024938762188, 'f1_score': 0.0019050287155063736, 'f1_score -> average@micro': 0.019420248464943595}


### API structures & references

>NOTE: docs 참고, 주요 구조 기록



#### introduction 
>The core motivation of DHG is to attach those spectral-based and spatial-based operations to each specified structure. When a structure is created, those related Laplacian Matrices and message passing operations with different aggregation functions can be called and combined to manipulate any input features. Currently, the DHG has implemented the following structures and attached operations. More structures and operations will be added in the future. Welcome to contribute!

- 무슨 소리냐 이게 다?
1. The core motivation of DHG is to attach those spectral-based and spatial-based operations to each specified structure. : spectral-based operation 과 spatial-based operation을 연결하고자 하는 동기에서 시작됨.
    - spectral-based operation & spatial-based operation?
        - [참고자료](https://thejb.ai/comprehensive-gnns-3/)
2. When a structure is created, those related Laplacian Matrices and message passing operations with different aggregation functions can be called and combined to manipulate any input features.
    - 관계된 Laplacian Matrix 와 message passing operation 이 dhg framework에서 제공하는 structure manipulation의 핵심.
        - message passing operation?
3. 아래 summary 에서 지원하고 있는 연산자들을 확인할 수 있다.


|![summary of supported structures and attached operations](https://user-images.githubusercontent.com/60145951/228912936-f55be4f5-b121-4d0e-a404-2cd427503a4b.png)|
|:-:|
|github : [https://github.com/iMoonLab/DeepHypergraph/blob/main/docs/source/start/structure.rst](https://github.com/iMoonLab/DeepHypergraph/blob/main/docs/source/start/structure.rst) |
- 그래프 표기 기호가 뭔가 했는데... `\mathcal{L}` 이었다... LaTeX의 영문 폰트 중 하나. (mathcal, mathbb, mathfrak, mathsf, mathbf) 가 있다. 그리스 문자 아니다. 근데 왜 L일까...
- 찾아본김에 확인한 그리스문자의 활용 : [wikipedia](https://en.wikipedia.org/wiki/Greek_letters_used_in_mathematics,_science,_and_engineering)
- `.rst` 는 뭘까? : *RST 파일은 주로 Python 프로그래밍 커뮤니티에서 사용되는 기술 문서 파일 형식입니다. 문서 생성을 위해 일반 텍스트 문서에 스타일과 서식을 적용하는 reStructuredText 마크업 언어로 작성된 텍스트 파일입니다. RST 파일은 Python 프로그램 코드의 주석 및 기타 정보를 사용하여 응용 프로그램의 기술 문서를 만듭니다.* [출처](https://docs.fileformat.com/ko/programming/rst/)


In [4]:
import torch
import dhg
g = dhg.random.graph_Gnm(5, 8)

In [8]:
# Generate a vertex feature matrix with size 5x2
X = torch.rand(5, 2)

# Print vertex features
X

tensor([[0.6883, 0.7243],
        [0.7425, 0.1050],
        [0.9149, 0.7520],
        [0.4753, 0.2716],
        [0.2399, 0.2793]])

In [11]:
# Print information about the graph and feature
print(f"information about the graph and feature : {g}")

# Print edges in the graph
print(f"\nedges in the graph: {g.e[0]}")

# Print the inside Laplacian Matrix by GCN on the graph structure
print(f"\nthe inside Laplacian Matrix by GCN on the graph structure : {g.L_GCN.to_dense()}")

# Print the vertex features after GCN-based smoothing
X_ = g.smoothing_with_GCN(X)
print(f"\nthe vertex features after GCN-based smoothing : {X_}")


information about the graph and feature : Graph(num_v=5, num_e=8)

edges in the graph: [(0, 1), (2, 4), (1, 2), (0, 4), (3, 4), (0, 3), (1, 4), (2, 3)]

the inside Laplacian Matrix by GCN on the graph structure : tensor([[0.2500, 0.2500, 0.0000, 0.2500, 0.2236],
        [0.2500, 0.2500, 0.2500, 0.0000, 0.2236],
        [0.0000, 0.2500, 0.2500, 0.2500, 0.2236],
        [0.2500, 0.0000, 0.2500, 0.2500, 0.2236],
        [0.2236, 0.2236, 0.2236, 0.2236, 0.2000]])

the vertex features after GCN-based smoothing : tensor([[0.5302, 0.3377],
        [0.6401, 0.4578],
        [0.5868, 0.3446],
        [0.5733, 0.4994],
        [0.6788, 0.4702]])


### Message Propagation

message propagation ([참고: Learning How to propagate messages in GNN](https://dl.acm.org/doi/10.1145/3447548.3467451))
- *Teng Xiao, Zhengyu Chen, Donglin Wang, and Suhang Wang. 2021. Learning How to Propagate Messages in Graph Neural Networks. In Proceedings of the 27th ACM SIGKDD Conference on Knowledge Discovery & Data Mining (KDD '21). Association for Computing Machinery, New York, NY, USA, 1894–1903. https://doi.org/10.1145/3447548.3467451*



### Expansion

cooking200 예제에서 expansion을 사용한 바 있다. dhg에서 지원하며 star, clique, hypergcn 방법을 사용할 수 있는데 이는  [공식문서](https://deephypergraph.readthedocs.io/en/latest/tutorial/structure.html#reduced-from-high-order-structures)에서 더 살펴볼 수 있다.

확인해야 하는 것은 해당 expansion의 변환 방법과 변환 결과로, 변환 결과를 알아야 GCN에 $A$로 입력값을 넣을 수 있고 GCN based model에서 다양한 topology를 적용해볼 수 있기 때문이다.
- 사실 이렇게되면 dhg 이용해서 expansion만 하면 끝나는거라 모델 구조 제안이나 hypergraph 적용에도 기여를 못하게 되는데 우선... 확인부터 하고

In [12]:
import dhg

In [ ]:
# clique expansion
# 모든 노드들이 엣지로 연결된 부분그래프를 클리크(clique)라고 합니다. 
# https://ratsgo.github.io/data%20structure&algorithm/2017/11/18/graph/